In [1]:
import numpy as np
import fabio
#import cupy as cp
from datetime import datetime
from numba import jit, cuda, prange

In [2]:
#initialze detector info
# //These are the details of the detector in the detInfo wave
# 	//detInfo[0] = xdim
# 	//detInfo[1] = ydim
# 	//detInfo[2] = xpixsize	Given in mm
# 	//detInfo[3] = ypixsize	Given in mm
# 	//detInfo[4] = xcen (pixels)
# 	//detInfo[5] = ycen (pixels)
# 	//detInfo[6] = tiltang	Given in degrees
# 	//detInfo[7] = tiltrot		Given in degrees
# 	//detInfo[8] = sample to detector distance	Given in mm
# 	//detInfo[9] = lamda	Given in Angstroms
# detInfo = np.array([1475,1679,0.17200001,0.17200001,738.00598,829.4032,0,0,450,0.23946001])
detInfo = np.array([2048,2048,0.085,0.085,1062.08,1042.1,-0.655953,133.021,714.029,0.16118])
imFilePrefix = ""
imNumbers = np.arange(1,1000)
boxsize = 501
qmax = 5
# boxname = ""
omegaPos = np.arange(9.8,-50.2,-0.2)
phiPos = np.zeros_like(omegaPos)
psiPos = np.zeros_like(omegaPos)
mask = []
rejects = []
cust_mask = []
mask_list = []
rotmatrix = np.array([0.61190927,0.098435618,-0.78477865,-0.78786123,-0.011431515,-0.61574668,0.069582619,-0.99507779,-0.070558563]).reshape((3,3))
normInt = np.arange(1,1000)
startIm = 0
# box = np.zeros((boxsize,boxsize,boxsize),dtype=np.float32)

In [3]:
def DiffuseReconstruction(fileNum):
    tiltang = detInfo[6]*np.pi/180 
    tiltrot = detInfo[7]*np.pi/180 
    dist = detInfo[8]
#     row,col = np.indices((int(detInfo[0]),int(detInfo[1])))
    row = np.arange(0,int(detInfo[0])).repeat(int(detInfo[1])).reshape(int(detInfo[0]),int(detInfo[1]))
    col = np.arange(0,int(detInfo[1])).repeat(int(detInfo[0])).reshape(int(detInfo[1]),int(detInfo[0]))
    col = np.transpose(col)
#     TT = (np.arctan(np.sqrt((np.cos(tiltang)**2*(np.cos(tiltrot)*((row-detInfo[4])*detInfo[2])+np.sin(tiltrot)*((col-detInfo[5])*detInfo[3]))**2 + (-np.sin(tiltrot)*((row-detInfo[4])*detInfo[2]) + np.cos(tiltrot)*((col-detInfo[5])*detInfo[3]))**2)/((dist+np.sin(tiltang)*(np.cos(tiltrot)*((row-detInfo[4])*detInfo[2])+np.sin(tiltrot)*((col-detInfo[5])*detInfo[3])))**2))))/2
    TT = (np.sqrt(np.arctan((np.cos(tiltang)**2*(np.cos(tiltrot)*((row-detInfo[4])*detInfo[2])+np.sin(tiltrot)*((col-detInfo[5])*detInfo[3]))**2 + (-np.sin(tiltrot)*((row-detInfo[4])*detInfo[2]) + np.cos(tiltrot)*((col-detInfo[5])*detInfo[3]))**2)/((dist+np.sin(tiltang)*(np.cos(tiltrot)*((row-detInfo[4])*detInfo[2])+np.sin(tiltrot)*((col-detInfo[5])*detInfo[3])))**2))))/2
    AZ = np.arctan2(((col-detInfo[5])*detInfo[3]), ((row-detInfo[4])*detInfo[2]))+np.pi 
    TT = np.transpose(TT)
    AZ = np.transpose(AZ)
    
    TT1D = np.reshape(TT,int(detInfo[0])*int(detInfo[1]))
    AZ1D = np.reshape(AZ,int(detInfo[0])*int(detInfo[1]))

#     boxname = np.empty((boxsize,boxsize,boxsize))
#     boxname[:] = np.NaN
#     numpixels = np.empty((boxsize,boxsize,boxsize))
#     numpixels[:] = np.NaN
#     boxname = np.zeros((boxsize,boxsize,boxsize))
#     numpixels = np.zeros((boxsize,boxsize,boxsize))

    bigbox = np.zeros((boxsize,boxsize,boxsize),dtype=np.float32)
    numpixels = np.zeros((boxsize,boxsize,boxsize),dtype=np.uint32)
    # Use ravel() for shallow copy or flatten() for deep copy

    BBposX = np.zeros(int(detInfo[0])*int(detInfo[1]),dtype=np.int16)
    BBposY = np.zeros(int(detInfo[0])*int(detInfo[1]),dtype=np.int16)
    BBposZ = np.zeros(int(detInfo[0])*int(detInfo[1]),dtype=np.int16)

    qrangex = np.linspace(-(qmax),qmax,boxsize)
    qrangey = np.linspace(-(qmax),qmax,boxsize)
    qrangez = np.linspace(-(qmax),qmax,boxsize)

    # for i in range(int(detInfo[0])*int(detInfo[1])):
    for i in range(fileNum):
        if i % 10 == 0:
            print(i)
        #use FABIO to load the image
        CI_fabio = fabio.open("C:\\Users\\Patrick\\Desktop\\s1_363K_2p0_diffuse\\DisCorr\\s1_363K_2p0_diffuse_"+("%04d" % (i))+".edf")
        #convert to numpy array
        CI = np.asarray(CI_fabio.data)
        CI = np.transpose(CI)
        CI = np.reshape(CI,int(detInfo[0])*int(detInfo[1]))
        
        bigboxloop,numpixelsloop = looper(i,CI,TT1D,AZ1D,qmax,qrangex,qrangey,qrangez,BBposX,BBposY,BBposZ)
        bigbox += bigboxloop
        numpixels += numpixelsloop
#     print(bigbox)
    
    #ignore dividing by inf or 0
    np.seterr(divide='ignore', invalid='ignore')
    bigbox /= numpixels
#     print(bigbox[253, 141, 250])
    
    return bigbox, numpixels
    
#     print(boxname[17, 656, 549])

In [5]:
@jit(nopython=True,parallel=True)
# @jit(nopython=True)
def looper(row,CI,TT1D,AZ1D,qmax,qrangex,qrangey,qrangez,BBposX,BBposY,BBposZ):
    
    bigboxloop = np.zeros((boxsize,boxsize,boxsize),dtype=np.float32)
    numpixelsloop = np.zeros((boxsize,boxsize,boxsize),dtype=np.uint32)
    
    omega = omegaPos[row]*np.pi/180
    phi = phiPos[row]*np.pi/180
    psi = psiPos[row]*np.pi/180

    qx = (4*np.pi*np.sin(TT1D)/detInfo[9]) * (np.sin(TT1D)*(np.sin(phi)*np.sin(psi)*np.sin(omega) + np.cos(phi)*np.cos(omega)) + np.cos(TT1D)*np.cos(AZ1D)*np.sin(phi)*np.cos(psi) - np.cos(TT1D)*np.sin(AZ1D)*(np.sin(phi)*np.sin(psi)*np.cos(omega) - np.cos(phi)*np.sin(omega)))
    qy = (4*np.pi*np.sin(TT1D)/detInfo[9]) * (-np.sin(TT1D)*(np.cos(phi)*np.sin(psi)*np.sin(omega) - np.sin(phi)*np.cos(omega)) - np.cos(TT1D)*np.cos(AZ1D)*np.cos(phi)*np.cos(psi) + np.cos(TT1D)*np.sin(AZ1D)*(np.cos(phi)*np.sin(psi)*np.cos(omega) + np.sin(phi)*np.sin(omega)))
    qz = (4*np.pi*np.sin(TT1D)/detInfo[9]) * (np.sin(TT1D)*np.cos(psi)*np.sin(omega) - np.cos(TT1D)*np.sin(AZ1D)*np.cos(psi)*np.cos(omega) - np.cos(TT1D)*np.cos(AZ1D)*np.sin(psi))
#         qz = (4*np.pi*np.sin(TT1D)/detInfo[9]) * (np.sin(TT1D)*np.cos(psi)*np.sin(omega) - np.cos(TT1D)*np.sin(AZ1D)*np.cos(psi)*np.cos(omega) - np.cos(TT1D)*np.cos(AZ1D)*np.sin(psi))
    
    ### maybe need to change this
#     qxyz = np.concatenate((qx,qy,qz)).reshape(qx.size,3)
#     print(qxyz)
#     qxyz = rotmatrix @ np.transpose(qxyz)
#     print(qxyz)
#     qxyz = np.transpose(qxyz)

    np.round((qx - (qrangex[0])) / (qrangex[1]-qrangex[0]),0,BBposX).astype(np.int64)
    np.round((qy - (qrangey[0])) / (qrangey[1]-qrangey[0]),0,BBposY).astype(np.int64)
    np.round((qz - (qrangez[0])) / (qrangez[1]-qrangez[0]),0,BBposZ).astype(np.int64)
    
    for j in range(CI.size):
        if(0 <= BBposX[j] and 0 <= BBposY[j] and 0 <= BBposZ[j] and BBposX[j] < bigboxloop.shape[0] and BBposY[j] < bigboxloop.shape[1] and BBposZ[j] < bigboxloop.shape[2] ):
#         if(0 > BBposX[j] or 0 > BBposY[j] or 0 > BBposZ[j] or BBposX[j] > bigboxloop.shape[0] or BBposY[j] > bigboxloop.shape[1] or BBposZ[j] > bigboxloop.shape[2] ):
            
            if(np.isfinite(CI[j])):
    #             print(j)
    #                 if(np.isnan(boxname[BBposX[j],BBposY[j],BBposZ[j]]) == True):
#                 BBX = BBposX[j]
#                 BBY = BBposY[j]
#                 BBZ = BBposZ[j]
#                 bigboxloop[BBX,BBY,BBZ] += CI[j]
#                 numpixelsloop[BBX,BBY,BBZ] += 1
                bigboxloop[BBposX[j],BBposY[j],BBposZ[j]] += CI[j]
                numpixelsloop[BBposX[j],BBposY[j],BBposZ[j]] += 1 
    
    return bigboxloop,numpixelsloop

In [6]:
import gc
#garbage collector
gc.collect()

box,numpixels = DiffuseReconstruction(100)

0
10
20
30
40
50
60
70
80
90


In [7]:
from matplotlib import pyplot       # Load matplotlib
%matplotlib qt
pyplot.imshow(box[:, 250, :]) 
pyplot.show() 


In [7]:
import winsound
winsound.Beep(300, 750)

In [6]:
box[253, 141, 250]

nan

In [64]:
# numpixels[17, 656, 549]
np.max(numpixels)

0

In [6]:
startTime = datetime.now()

box = DiffuseReconstruction(100)

print(datetime.now() - startTime)

0
10
20
30
40
50
60
70
80
90
0:01:24.638224
